In [31]:
import sys
sys.path.append('../src')

In [32]:
import os
import pickle

In [33]:
import numpy as np
import soundfile as sf

In [34]:
from soundgenerator import SoundGenerator
from autoencoder import VAE

In [35]:
from dotenv import load_dotenv
load_dotenv()

False

In [36]:
HOP_LENGTH = 256
data_dir            = os.getenv('DATA_DIR')
SPECTROGRAMS_PATH   = os.path.join(data_dir, 'spectrograms')
MIN_MAX_VALUES_PATH = os.path.join(data_dir, 'fsdd/min_max_values.pkl') 
SAVE_DIR_ORIGINAL   = os.path.join(data_dir, 'original')
SAVE_DIR_GENERATED  = os.path.join(data_dir, 'generated')
MODEL_DIR           = os.path.join(data_dir, 'model')

In [37]:
os.makedirs(SAVE_DIR_ORIGINAL, exist_ok=True)
os.makedirs(SAVE_DIR_GENERATED, exist_ok=True)

In [38]:
def load_fsdd(spectrograms_path):
    x_train = []
    file_paths = []
    for root, _, file_names in os.walk(spectrograms_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames, 1)
            x_train.append(spectrogram)
            file_paths.append(file_path)
    x_train = np.array(x_train)
    x_train = x_train[..., np.newaxis] # -> (3000, 256, 64, 1)
    return x_train, file_paths

In [39]:
def select_spectrograms(spectrograms,
                        file_paths,
                        min_max_values,
                        num_spectrograms=2):
    sampled_indexes = np.random.choice(range(len(spectrograms)), num_spectrograms)
    sampled_spectrogrmas = spectrograms[sampled_indexes]
    for p in file_paths:
        print(p)
    file_paths = [file_paths[index] for index in sampled_indexes]
    sampled_min_max_values = [min_max_values[file_path] for file_path in
                           file_paths]
    return sampled_spectrogrmas, sampled_min_max_values

In [40]:
def save_signals(signals, save_dir, sample_rate=22050):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for i, signal in enumerate(signals):
        save_path = os.path.join(save_dir, str(i) + ".wav")
        sf.write(save_path, signal, sample_rate)

In [41]:
def test_generate():
    # initialise sound generator
    vae = VAE.load(MODEL_DIR)
    sound_generator = SoundGenerator(vae, HOP_LENGTH)

    # load spectrograms + min max values
    with open(MIN_MAX_VALUES_PATH, "rb") as f:
        min_max_values = pickle.load(f)

    specs, file_paths = load_fsdd(SPECTROGRAMS_PATH)

    # sample spectrograms + min max values
    sampled_specs, sampled_min_max_values = select_spectrograms(specs,
                                                                file_paths,
                                                                min_max_values,
                                                                5)

    # generate audio for sampled spectrograms
    signals, _ = sound_generator.generate(sampled_specs,
                                          sampled_min_max_values)

    # convert spectrogram samples to audio
    original_signals = sound_generator.convert_spectrograms_to_audio(
        sampled_specs, sampled_min_max_values)

    # save audio signals
    save_signals(signals, SAVE_DIR_GENERATED)
    save_signals(original_signals, SAVE_DIR_ORIGINAL)

In [42]:
def compare_signals(signals):
    for i, signal in enumerate(signals):
        print(f'Signal {i+1}: Mean={np.mean(signal)}, StdDev={np.std(signal)}')

In [43]:
def generate_unique():
    vae = VAE.load(MODEL_DIR)
    sound_generator = SoundGenerator(vae, HOP_LENGTH)
    signals = sound_generator.generate_unique(5)
    compare_signals(signals)
    save_signals(signals, SAVE_DIR_GENERATED)

In [44]:
# test_generate()

In [46]:
generate_unique()
   

random_seed (1, 128)


/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


random_seed (1, 128)
random_seed (1, 128)
random_seed (1, 128)
random_seed (1, 128)
Signal 1: Mean=1.229337249242235e-05, StdDev=0.0010931624565273523
Signal 2: Mean=1.2336155123193748e-05, StdDev=0.0011248858645558357
Signal 3: Mean=1.2279801921977196e-05, StdDev=0.0010688917245715857
Signal 4: Mean=1.2304808478802443e-05, StdDev=0.0011033909395337105
Signal 5: Mean=1.232097656611586e-05, StdDev=0.0011184848845005035
